## Early Stopping
Early stopping is a method to avoid overfitting the training set. The general idea is interrupting the training process when its performance on the validation set starts dropping.

A simple way to implement Early Stopping in TensorFlow is to evaluate the model on a validation set at regular intervals (for example, every 50 steps), and save the best model if the current model outperforms the previous best one. Count the number of steps since the last best snapshot was saved, and interrupt training when this number reaches a limit.

Early Stopping presents better performance when using with other regularization techniques.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()
n_inputs = 784
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.elu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xen = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xen, name="loss")

In [2]:
with tf.name_scope("train"):
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss, global_step=global_step)

In [3]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [7]:
init = tf.global_variables_initializer()

n_epochs = 50
batch_size = 100

best_epoch = None # Store the epoch with the highest accuracy score
best_accuracy = 0.95 # Best accuracy threshold

mnist = input_data.read_data_sets("/tmp/data/", validation_size=6000)

saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(len(mnist.test.labels) // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        # Evaluate the model every 50 epochs on the validation set
        # And store the best model if available
        # if epoch % 5 == 0:
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print("Epoch:", epoch, "--", "Validation Accuracy:", acc_val)
        if acc_val > best_accuracy:
            best_accuracy = acc_val
            best_epoch = epoch
            saver.save(sess, "models/early_stopping/best_model.cpkt")
    print("Best Epoch:", best_epoch)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch: 0 -- Validation Accuracy: 0.9205
Epoch: 1 -- Validation Accuracy: 0.928167
Epoch: 2 -- Validation Accuracy: 0.952667
Epoch: 3 -- Validation Accuracy: 0.9445
Epoch: 4 -- Validation Accuracy: 0.9575
Epoch: 5 -- Validation Accuracy: 0.959
Epoch: 6 -- Validation Accuracy: 0.966833
Epoch: 7 -- Validation Accuracy: 0.9635
Epoch: 8 -- Validation Accuracy: 0.968667
Epoch: 9 -- Validation Accuracy: 0.968667
Epoch: 10 -- Validation Accuracy: 0.974333
Epoch: 11 -- Validation Accuracy: 0.972667
Epoch: 12 -- Validation Accuracy: 0.967
Epoch: 13 -- Validation Accuracy: 0.974833
Epoch: 14 -- Validation Accuracy: 0.9725
Epoch: 15 -- Validation Accuracy: 0.975833
Epoch: 16 -- Validation Accuracy: 0.975833
Epoch: 17 -- Validation Accuracy: 0.975167
Epoch: 18 -- Validation Accuracy: 0.977333
Epoch: 19 -- Valida

In [10]:
from sklearn.metrics import accuracy_score

# Restore the best model and make predictions
with tf.Session() as sess:
    saver.restore(sess, "models/early_stopping/best_model.cpkt")
    X_new_scaled = mnist.test.images[:30]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

    print("Predicted classes:", y_pred)
    print("Actual classes   :", mnist.test.labels[:30])
    print(accuracy_score(mnist.test.labels[:30], y_pred))

INFO:tensorflow:Restoring parameters from models/early_stopping/best_model.cpkt
[[ -2.51853991   1.08582044   1.5616163    7.22020721  -9.50950432
   -3.73617554  -9.01093388  15.74015522  -3.00265861   3.07920527]
 [  0.6129775   10.28488541  26.42911148   5.81304073 -11.73921967
   -0.18888301   0.61764717  -7.79584026  -1.73719966 -19.26265335]
 [ -3.13051486  13.50319576   1.41257191  -3.43775916  -0.76488507
   -5.24222326  -0.36236531   4.75409317   3.10113931  -8.68609905]
 [ 15.61366463  -6.77007484   2.39362192  -5.58926535  -6.12625551
    2.73630071   0.81752777  -2.13054991 -10.53195572  -0.92315429]
 [ -4.73786783  -5.75126553  -5.48402357  -6.6187048   19.08856201
   -7.31355095  -3.13586926   2.47816038  -3.82513142   8.68794155]
 [ -1.55765939  17.8956852    0.45966649  -3.93115544   2.69311619
   -9.62004757  -2.92325091   9.11577892   1.29965532  -9.57527065]
 [ -8.00997925  -2.34052086  -3.2098434   -6.23173523  15.72913361
   -2.66440678  -2.5109787    2.93699384   